In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set dataset path (adjust if your folder is in a different location)
DATA_DIR = '/content/drive/MyDrive/Tuberculosis-Detection/Dataset'

# Verify dataset exists
if os.path.exists(DATA_DIR):
    print(f"✓ Dataset found at: {DATA_DIR}")
    print(f"  - Normal images: {len(os.listdir(os.path.join(DATA_DIR, 'Normal Chest X-rays')))}")
    print(f"  - TB images: {len(os.listdir(os.path.join(DATA_DIR, 'TB Chest X-rays')))}")
else:
    print(f"✗ Dataset NOT found at: {DATA_DIR}")
    print("Please upload your dataset to Google Drive first!")

## Check GPU Availability

In [ ]:
import tensorflow as tf

print("=" * 70)
print("GPU CONFIGURATION")
print("=" * 70)

# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# Check GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ GPU Available: {len(gpus)} device(s)")
    for gpu in gpus:
        print(f"  - {gpu.name}")
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPU memory growth enabled")
else:
    print("✗ No GPU detected. Using CPU.")
    print("To enable GPU: Runtime → Change runtime type → GPU")

print("=" * 70)

## Install Additional Dependencies

In [ ]:
# Colab already has most packages, but let's ensure we have everything
!pip install -q opencv-python-headless

print("✓ All dependencies installed!")

## Import Required Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import mixed_precision
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

# Enable mixed precision for faster training
mixed_precision.set_global_policy('mixed_float16')
print("✓ Mixed precision enabled (float16)")

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print("✓ All libraries imported successfully!")

## Configuration Parameters

In [ ]:
# Configuration
class Config:
    # Dataset paths
    DATA_DIR = '/content/drive/MyDrive/Tuberculosis-Detection/Dataset'
    NORMAL_DIR = os.path.join(DATA_DIR, "Normal Chest X-rays")
    TB_DIR = os.path.join(DATA_DIR, "TB Chest X-rays")

    # Output directories
    OUTPUT_DIR = "/content/outputs"
    MODEL_DIR = os.path.join(OUTPUT_DIR, "models")
    PLOT_DIR = os.path.join(OUTPUT_DIR, "plots")

    # Image parameters
    IMG_SIZE = (224, 224)
    IMG_HEIGHT, IMG_WIDTH = IMG_SIZE
    CHANNELS = 3

    # Training parameters
    BATCH_SIZE = 16  # Reduced for memory efficiency
    EPOCHS = 30  # Increased for better learning
    LEARNING_RATE = 2e-4  # Slightly higher initial learning rate

    # Data split ratios
    TRAIN_RATIO = 0.70
    VAL_RATIO = 0.15
    TEST_RATIO = 0.15

    # Model parameters
    L2_REG = 1e-4

    # Callbacks
    EARLY_STOPPING_PATIENCE = 8
    LR_REDUCE_PATIENCE = 3
    LR_REDUCE_FACTOR = 0.5

    RANDOM_SEED = RANDOM_SEED

# Create output directories
os.makedirs(Config.MODEL_DIR, exist_ok=True)
os.makedirs(Config.PLOT_DIR, exist_ok=True)

print("✓ Configuration set!")
print(f"  - Image size: {Config.IMG_SIZE}")
print(f"  - Batch size: {Config.BATCH_SIZE}")
print(f"  - Epochs: {Config.EPOCHS}")
print(f"  - Learning rate: {Config.LEARNING_RATE}")

## Data Loading and Preprocessing (Memory Efficient)

In [ ]:
def collect_image_paths(normal_dir, tb_dir):
    """
    Collect file paths instead of loading images into memory
    MEMORY EFFICIENT - Only stores paths, not image data
    """
    print("\n" + "=" * 70)
    print("COLLECTING IMAGE PATHS (Memory Efficient)")
    print("=" * 70)

    file_paths = []
    labels = []

    # Collect Normal image paths (label = 0)
    if os.path.exists(normal_dir):
        normal_files = [f for f in os.listdir(normal_dir)
                       if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Found {len(normal_files)} normal images")

        for filename in normal_files:
            img_path = os.path.join(normal_dir, filename)
            file_paths.append(img_path)
            labels.append(0)

    # Collect TB-positive image paths (label = 1)
    if os.path.exists(tb_dir):
        tb_files = [f for f in os.listdir(tb_dir)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        print(f"Found {len(tb_files)} TB-positive images")

        for filename in tb_files:
            img_path = os.path.join(tb_dir, filename)
            file_paths.append(img_path)
            labels.append(1)

    labels = np.array(labels)

    print(f"\n✓ Total images found: {len(file_paths)}")
    print(f"  - Normal: {np.sum(labels == 0)}")
    print(f"  - TB-positive: {np.sum(labels == 1)}")
    print(f"\n✓ Memory usage: Minimal (only paths stored, not images)")

    return file_paths, labels

# Collect paths only (not loading images yet)
file_paths, labels = collect_image_paths(
    Config.NORMAL_DIR,
    Config.TB_DIR
)

## Visualize Sample Images

In [ ]:
def load_and_display_samples(file_paths, labels, num_samples=5):
    """
    Load and display only a few sample images (memory efficient)
    """
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 6))

    # Get indices for each class
    normal_indices = np.where(labels == 0)[0][:num_samples]
    tb_indices = np.where(labels == 1)[0][:num_samples]

    # Load and display normal samples
    for i, idx in enumerate(normal_indices):
        img = cv2.imread(file_paths[idx], cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, Config.IMG_SIZE)
            axes[0, i].imshow(img, cmap='gray')
            axes[0, i].set_title('Normal', fontsize=10)
            axes[0, i].axis('off')

    # Load and display TB samples
    for i, idx in enumerate(tb_indices):
        img = cv2.imread(file_paths[idx], cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, Config.IMG_SIZE)
            axes[1, i].imshow(img, cmap='gray')
            axes[1, i].set_title('TB Positive', fontsize=10)
            axes[1, i].axis('off')

    plt.tight_layout()
    plt.savefig(os.path.join(Config.PLOT_DIR, 'sample_images.png'), dpi=150)
    plt.show()

    print("✓ Sample images visualized!")

# Display samples
load_and_display_samples(file_paths, labels)

## Split Dataset (Patient-Level)

In [ ]:
def split_dataset_paths(file_paths, labels, train_ratio, val_ratio, test_ratio):
    """
    Split file paths (not loaded images) to avoid memory issues
    """
    print("\n" + "=" * 70)
    print("SPLITTING DATASET (PATIENT-LEVEL)")
    print("=" * 70)

    # Convert to arrays
    file_paths = np.array(file_paths)

    # First split: separate test set
    train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
        file_paths,
        labels,
        test_size=test_ratio,
        random_state=Config.RANDOM_SEED,
        stratify=labels
    )

    # Second split: separate train and validation
    val_ratio_adjusted = val_ratio / (train_ratio + val_ratio)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        train_val_paths,
        train_val_labels,
        test_size=val_ratio_adjusted,
        random_state=Config.RANDOM_SEED,
        stratify=train_val_labels
    )

    print(f"✓ Training set: {len(train_paths)} images")
    print(f"    Normal: {np.sum(train_labels==0)}, TB: {np.sum(train_labels==1)}")
    print(f"✓ Validation set: {len(val_paths)} images")
    print(f"    Normal: {np.sum(val_labels==0)}, TB: {np.sum(val_labels==1)}")
    print(f"✓ Test set: {len(test_paths)} images")
    print(f"    Normal: {np.sum(test_labels==0)}, TB: {np.sum(test_labels==1)}")

    return {
        'train': (train_paths, train_labels),
        'val': (val_paths, val_labels),
        'test': (test_paths, test_labels)
    }

# Split the data (paths only)
data_splits = split_dataset_paths(
    file_paths, labels,
    Config.TRAIN_RATIO, Config.VAL_RATIO, Config.TEST_RATIO
)

train_paths, train_labels = data_splits['train']
val_paths, val_labels = data_splits['val']
test_paths, test_labels = data_splits['test']

# Save test paths for later
test_data = {'paths': test_paths, 'labels': test_labels}
test_data_path = os.path.join(Config.OUTPUT_DIR, 'test_data.npz')
np.savez(test_data_path, **test_data)
print(f"\n✓ Test data paths saved to: {test_data_path}")

## Data Augmentation (Memory Efficient with tf.data)

In [ ]:
def load_and_preprocess_image(file_path, label, img_size, augment=False):
    """
    Load and preprocess a single image (called by tf.data pipeline)
    """
    # Read image file
    img = tf.io.read_file(file_path)

    # Try multiple decoders for compatibility
    try:
        img = tf.image.decode_jpeg(img, channels=3)
    except:
        try:
            img = tf.image.decode_png(img, channels=3)
        except:
            img = tf.image.decode_image(img, channels=3)

    # Resize
    img = tf.image.resize(img, img_size)

    # Normalize to [0, 1]
    img = tf.cast(img, tf.float32) / 255.0

    if augment:
        # Random augmentations (all native TensorFlow operations)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.15)
        img = tf.image.random_contrast(img, 0.85, 1.15)
        img = tf.image.random_saturation(img, 0.9, 1.1)

        # Random zoom simulation (crop and resize)
        if tf.random.uniform([]) > 0.5:
            crop_size = tf.random.uniform([], 0.85, 1.0)
            crop_h = tf.cast(img_size[0] * crop_size, tf.int32)
            crop_w = tf.cast(img_size[1] * crop_size, tf.int32)
            img = tf.image.random_crop(img, [crop_h, crop_w, 3])
            img = tf.image.resize(img, img_size)

    return img, label

def create_tf_dataset(file_paths, labels, batch_size, img_size, augment=False, shuffle=True):
    """
    Create memory-efficient tf.data.Dataset pipeline
    MEMORY EFFICIENT - Loads images on-the-fly during training
    """
    # Create dataset from file paths
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000, seed=Config.RANDOM_SEED)

    # Map loading and preprocessing (parallel processing)
    dataset = dataset.map(
        lambda x, y: load_and_preprocess_image(x, y, img_size, augment),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Batch and prefetch
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset

print("\n" + "=" * 70)
print("CREATING MEMORY-EFFICIENT DATA PIPELINE")
print("=" * 70)

# Create datasets (no tensorflow-addons needed!)
train_dataset = create_tf_dataset(
    train_paths, train_labels,
    Config.BATCH_SIZE, Config.IMG_SIZE,
    augment=True, shuffle=True
)

val_dataset = create_tf_dataset(
    val_paths, val_labels,
    Config.BATCH_SIZE, Config.IMG_SIZE,
    augment=False, shuffle=False
)

test_dataset = create_tf_dataset(
    test_paths, test_labels,
    Config.BATCH_SIZE, Config.IMG_SIZE,
    augment=False, shuffle=False
)

print("✓ Memory-efficient pipeline created!")
print("  - Images loaded on-the-fly (not stored in RAM)")
print("  - Training augmentation: flip, brightness, contrast, saturation, zoom")
print("  - Parallel loading with prefetching")
print(f"  - Batch size: {Config.BATCH_SIZE}")
print("  - Using native TensorFlow ops (no external dependencies)")

## Build Custom CNN Model (From Scratch - Enhanced)

In [ ]:
def build_tb_cnn_model(input_shape, l2_reg=1e-4):
    """
    Build ENHANCED custom CNN from scratch for TB detection
    NO TRANSFER LEARNING - Built entirely from scratch
    IMPROVED ARCHITECTURE for better accuracy (70-80%+)
    """
    print("\n" + "=" * 70)
    print("BUILDING ENHANCED CNN MODEL FROM SCRATCH")
    print("=" * 70)

    model = models.Sequential(name='TB_CNN_Detector_Enhanced')

    # Block 1
    model.add(layers.Input(shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.25))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(128, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.3))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.4))

    # Block 4
    model.add(layers.Conv2D(512, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(512, (3, 3), padding='same',
                           kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Dropout(0.5))

    # Global Average Pooling
    model.add(layers.GlobalAveragePooling2D())

    # Dense layers with more capacity
    model.add(layers.Dense(512, activation='relu',
                          kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(256, activation='relu',
                          kernel_regularizer=regularizers.l2(l2_reg)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid', dtype='float32'))

    print("✓ ENHANCED Model architecture:")
    print("  - 4 Convolutional Blocks (64->128->256->512 filters)")
    print("  - Extra conv layer in Block 3 for deeper features")
    print("  - Progressive Dropout: 0.25 → 0.5")
    print("  - L2 Regularization: 1e-4")
    print("  - 2 Dense layers (512 + 256) for better classification")
    print("  - Batch Normalization throughout")
    print("  - Global Average Pooling")
    print("  - Binary Classification Output")
    print("\n  EXPECTED ACCURACY: 70-85% with proper training")

    return model

# Build model
input_shape = (Config.IMG_HEIGHT, Config.IMG_WIDTH, Config.CHANNELS)
model = build_tb_cnn_model(input_shape, Config.L2_REG)

# Display model summary
model.summary()

## Compile Model

In [ ]:
# Calculate class weights for imbalanced data
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

print("Class Weights:")
print(f"  - Normal: {class_weight_dict[0]:.4f}")
print(f"  - TB: {class_weight_dict[1]:.4f}")

# Compile model
optimizer = keras.optimizers.Adam(learning_rate=Config.LEARNING_RATE)
loss = keras.losses.BinaryCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(curve='PR', name='auc_pr')
    ]
)

print("\n✓ Model compiled!")
print(f"  - Optimizer: Adam (lr={Config.LEARNING_RATE})")
print(f"  - Loss: Binary Cross-Entropy")
print(f"  - Metrics: Accuracy, Precision, Recall, AUC, AUC-PR")

## Setup Callbacks

In [ ]:
callbacks = [
    # Early stopping
    EarlyStopping(
        monitor='val_auc',
        patience=Config.EARLY_STOPPING_PATIENCE,
        verbose=1,
        mode='max',
        restore_best_weights=True
    ),

    # Reduce learning rate
    ReduceLROnPlateau(
        monitor='val_auc',
        factor=Config.LR_REDUCE_FACTOR,
        patience=Config.LR_REDUCE_PATIENCE,
        verbose=1,
        mode='max',
        min_lr=1e-7
    ),

    # Model checkpoint
    ModelCheckpoint(
        filepath=os.path.join(Config.MODEL_DIR, 'best_tb_cnn_model.h5'),
        monitor='val_auc',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

print("✓ Callbacks configured:")
print(f"  - EarlyStopping: patience={Config.EARLY_STOPPING_PATIENCE}")
print(f"  - ReduceLROnPlateau: factor={Config.LR_REDUCE_FACTOR}")
print(f"  - ModelCheckpoint: best_tb_cnn_model.h5")

## Train the Model

**This will take 30-60 minutes with GPU enabled.**

You can monitor:
- Loss decreasing
- Accuracy increasing
- **AUC (primary metric)** - should reach > 0.80
- **Recall (Sensitivity)** - Critical for TB detection

**MEMORY EFFICIENT:** Images loaded on-the-fly, not stored in RAM

In [ ]:
print("\n" + "=" * 70)
print("STARTING TRAINING (MEMORY EFFICIENT)")
print("=" * 70)
print(f"Training on: {len(train_paths)} images")
print(f"Validating on: {len(val_paths)} images")
print(f"Batch size: {Config.BATCH_SIZE}")
print(f"Epochs: {Config.EPOCHS}")
print(f"Learning rate: {Config.LEARNING_RATE}")
print("=" * 70)

# Calculate steps per epoch
steps_per_epoch = len(train_paths) // Config.BATCH_SIZE
validation_steps = len(val_paths) // Config.BATCH_SIZE

print(f"\nSteps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

# Train the model with tf.data pipeline (memory efficient)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=Config.EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n" + "=" * 70)
print("✓ TRAINING COMPLETED!")
print("=" * 70)

## Plot Training History

In [ ]:
# Plot training history
metrics_to_plot = ['loss', 'accuracy', 'precision', 'recall', 'auc']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, metric in enumerate(metrics_to_plot):
    if metric in history.history:
        axes[idx].plot(history.history[metric], label=f'Train {metric}')
        axes[idx].plot(history.history[f'val_{metric}'], label=f'Val {metric}')
        axes[idx].set_title(f'{metric.capitalize()} vs Epochs')
        axes[idx].set_xlabel('Epoch')
        axes[idx].set_ylabel(metric.capitalize())
        axes[idx].legend()
        axes[idx].grid(True)

plt.tight_layout()
plot_path = os.path.join(Config.PLOT_DIR, 'training_history.png')
plt.savefig(plot_path, dpi=300)
plt.show()

print(f"✓ Training history saved to: {plot_path}")

## Save Final Model

In [17]:
# Save final model
final_model_path = os.path.join(Config.MODEL_DIR, 'tb_cnn_model.h5')
model.save(final_model_path)
print(f"✓ Final model saved: {final_model_path}")

# Save model architecture as JSON
json_path = os.path.join(Config.MODEL_DIR, 'model_architecture.json')
with open(json_path, 'w') as f:
    f.write(model.to_json())
print(f"✓ Model architecture saved: {json_path}")

# Save training configuration
config_dict = {
    'img_size': Config.IMG_SIZE,
    'batch_size': Config.BATCH_SIZE,
    'epochs': Config.EPOCHS,
    'learning_rate': Config.LEARNING_RATE,
    'l2_regularization': Config.L2_REG,
}
config_path = os.path.join(Config.MODEL_DIR, 'training_config.json')
with open(config_path, 'w') as f:
    json.dump(config_dict, f, indent=4)
print(f"✓ Training config saved: {config_path}")

✓ Final model saved: /content/outputs/models/tb_cnn_model.h5
✓ Model architecture saved: /content/outputs/models/model_architecture.json
✓ Training config saved: /content/outputs/models/training_config.json


## Download Trained Model to Your Computer

Run this cell to download the trained model and outputs to your local machine.

In [ ]:
# Create a zip file of all outputs
!cd /content && zip -r outputs.zip outputs/

# Download the zip file
from google.colab import files
files.download('/content/outputs.zip')

print("✓ Outputs downloaded!")
print("Extract the zip file on your computer to access:")
print("  - best_tb_cnn_model.h5 (best model)")
print("  - tb_cnn_model.h5 (final model)")
print("  - training_history.png")
print("  - test_data.npz")

## Quick Evaluation on Test Set

In [ ]:
# Load best model
best_model = keras.models.load_model(os.path.join(Config.MODEL_DIR, 'best_tb_cnn_model.h5'))

# Evaluate on test set
print("\n" + "=" * 70)
print("EVALUATING ON TEST SET")
print("=" * 70)

# Evaluate with tf.data pipeline
test_results = best_model.evaluate(test_dataset, verbose=1)

print("\nTest Results:")
print(f"  - Loss: {test_results[0]:.4f}")
print(f"  - Accuracy: {test_results[1]:.4f}")
print(f"  - Precision: {test_results[2]:.4f}")
print(f"  - Recall (Sensitivity): {test_results[3]:.4f} ← PRIMARY METRIC")
print(f"  - AUC: {test_results[4]:.4f}")
print(f"  - AUC-PR: {test_results[5]:.4f}")

# Make predictions (load images for confusion matrix)
print("\nGenerating predictions for confusion matrix...")
y_pred_proba = best_model.predict(test_dataset).flatten()
y_pred = (y_pred_proba > 0.5).astype(int)

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(test_labels, y_pred)

print("\nConfusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(test_labels, y_pred, target_names=['Normal', 'TB']))

## Training Complete!

### Next Steps:

1. **Download your trained model** (cell 16)
2. **Run evaluation script** on your local machine: `python evaluate_model.py`
3. **Generate Grad-CAM visualizations**: `python gradcam_visualization.py`
4. **Convert to TFLite**: `python convert_to_tflite.py`

### Key Files in outputs.zip:
- `best_tb_cnn_model.h5` - Best model based on validation AUC
- `tb_cnn_model.h5` - Final model after all epochs
- `test_data.npz` - Test set for evaluation
- `training_history.png` - Training curves

---

# CONTINUE TRAINING YOUR MODEL

Run this cell to load your trained model and continue training for more epochs.

In [ ]:
print("=" * 70)
print("CONTINUE TRAINING FROM EXISTING MODEL")
print("=" * 70)

# 1. Load your already trained model
model_path = os.path.join(Config.MODEL_DIR, 'best_tb_cnn_model.h5')
loaded_model = keras.models.load_model(model_path)
print(f"✓ Model loaded from: {model_path}")

# 2. Set new training parameters (you can adjust these)
ADDITIONAL_EPOCHS = 10      # Train for 10 more epochs
NEW_LEARNING_RATE = 1e-4    # Lower learning rate (optional)

# 3. Recompile with new (or same) learning rate
loaded_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=NEW_LEARNING_RATE),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(curve='PR', name='auc_pr')
    ]
)

print(f"✓ Model recompiled with learning rate: {NEW_LEARNING_RATE}")
print(f"✓ Will train for {ADDITIONAL_EPOCHS} more epochs")

# 4. Setup callbacks
callbacks_continued = [
    EarlyStopping(
        monitor='val_auc',
        patience=5,
        verbose=1,
        mode='max',
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_auc',
        factor=0.5,
        patience=2,
        verbose=1,
        mode='max',
        min_lr=1e-7
    ),
    ModelCheckpoint(
        filepath=os.path.join(Config.MODEL_DIR, 'continued_tb_model.h5'),
        monitor='val_auc',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

print("=" * 70)
print("STARTING CONTINUED TRAINING")
print("=" * 70)

# 5. Continue training!
history_continued = loaded_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=ADDITIONAL_EPOCHS,
    steps_per_epoch=len(train_paths) // Config.BATCH_SIZE,
    validation_steps=len(val_paths) // Config.BATCH_SIZE,
    callbacks=callbacks_continued,
    class_weight=class_weight_dict,
    verbose=1
)

print("\n" + "=" * 70)
print("✓ CONTINUED TRAINING COMPLETED!")
print("=" * 70)
print(f"✓ New model saved to: continued_tb_model.h5")
print(f"✓ You can now evaluate this model on the test set")

# Quick comparison
print("\n📊 Quick Validation Results:")
val_results = loaded_model.evaluate(val_dataset, verbose=0)
print(f"  - Val Loss: {val_results[0]:.4f}")
print(f"  - Val Accuracy: {val_results[1]:.4f}")
print(f"  - Val AUC: {val_results[4]:.4f}")

---

## Continue Training ANY Model (Iterative Retraining)

Use this cell to keep training **any** model you've already trained.
- Train `continued_tb_model.h5` again
- Train the original model again
- Train any other saved model

Just update the `MODEL_TO_LOAD` variable!

In [ ]:
print("=" * 70)
print("ITERATIVE RETRAINING - TRAIN ANY MODEL AGAIN")
print("=" * 70)

# 📝 CHANGE THIS to train different models:
# Options: 'best_tb_cnn_model.h5', 'continued_tb_model.h5', 'tb_cnn_model.h5', etc.
MODEL_TO_LOAD = 'continued_tb_model_v3.h5'  # ← Change this to train a different model

# Training configuration (adjust these as needed)
EPOCHS_THIS_ROUND = 10           # How many more epochs to train
LEARNING_RATE_THIS_ROUND = 5e-5  # Lower LR for each iteration (5e-5, then 1e-5, etc.)
OUTPUT_MODEL_NAME = 'continued_tb_model_v4.h5'  # ← Name for the output model

print(f"Loading model: {MODEL_TO_LOAD}")
print(f"Will train for: {EPOCHS_THIS_ROUND} epochs")
print(f"Learning rate: {LEARNING_RATE_THIS_ROUND}")
print(f"Output will be saved as: {OUTPUT_MODEL_NAME}")
print("=" * 70)

# Load the specified model
model_path = os.path.join(Config.MODEL_DIR, MODEL_TO_LOAD)

if not os.path.exists(model_path):
    print(f"\n❌ ERROR: Model '{MODEL_TO_LOAD}' not found at {model_path}")
    print("\nAvailable models in your directory:")
    if os.path.exists(Config.MODEL_DIR):
        model_files = [f for f in os.listdir(Config.MODEL_DIR) if f.endswith('.h5')]
        for i, mf in enumerate(model_files, 1):
            print(f"   {i}. {mf}")
    else:
        print("   (Model directory not found)")
else:
    # Load model
    model_to_retrain = keras.models.load_model(model_path)
    print(f"\n✓ Model loaded successfully from: {MODEL_TO_LOAD}")

    # Recompile with new learning rate
    model_to_retrain.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_THIS_ROUND),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            keras.metrics.BinaryAccuracy(name='accuracy'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc'),
            keras.metrics.AUC(curve='PR', name='auc_pr')
        ]
    )
    print(f"✓ Recompiled with learning rate: {LEARNING_RATE_THIS_ROUND}")

    # Setup callbacks
    callbacks_iterative = [
        EarlyStopping(
            monitor='val_auc',
            patience=5,
            verbose=1,
            mode='max',
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.5,
            patience=2,
            verbose=1,
            mode='max',
            min_lr=1e-7
        ),
        ModelCheckpoint(
            filepath=os.path.join(Config.MODEL_DIR, OUTPUT_MODEL_NAME),
            monitor='val_auc',
            save_best_only=True,
            mode='max',
            verbose=1
        )
    ]

    print("\n" + "=" * 70)
    print("STARTING ITERATIVE TRAINING")
    print("=" * 70)

    # Train!
    history_iterative = model_to_retrain.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS_THIS_ROUND,
        steps_per_epoch=len(train_paths) // Config.BATCH_SIZE,
        validation_steps=len(val_paths) // Config.BATCH_SIZE,
        callbacks=callbacks_iterative,
        class_weight=class_weight_dict,
        verbose=1
    )

    print("\n" + "=" * 70)
    print("✓ ITERATIVE TRAINING COMPLETED!")
    print("=" * 70)
    print(f"✓ Model saved as: {OUTPUT_MODEL_NAME}")

    # Show results
    print("\n📊 Validation Results After This Round:")
    val_results = model_to_retrain.evaluate(val_dataset, verbose=0)
    print(f"  - Val Loss: {val_results[0]:.4f}")
    print(f"  - Val Accuracy: {val_results[1]:.4f}")
    print(f"  - Val Precision: {val_results[2]:.4f}")
    print(f"  - Val Recall: {val_results[3]:.4f}")
    print(f"  - Val AUC: {val_results[4]:.4f}")

    print("\n💡 To train this model again:")
    print(f"   1. Change MODEL_TO_LOAD to '{OUTPUT_MODEL_NAME}'")
    print(f"   2. Lower the learning rate (e.g., 1e-5 or 5e-6)")
    print(f"   3. Change OUTPUT_MODEL_NAME to 'continued_tb_model_v3.h5'")
    print("   4. Run this cell again!")